In [3]:
import networkx as nx
from deeprobust.graph.data import Dataset
import numpy as np
from scipy.sparse import csr_matrix
#from networkx import convert_matrix
import scipy.sparse
import matplotlib.pyplot as plt
import csv
import pandas as pd #pour lire/ecrire les csv et excel

In [4]:
#load data and preprocess
data = Dataset(root='D:/Docs/UQAM\Memoire/cryptograph/cryptograph-utility/code', name="polblogs", setting="gcn")
groundtruth_adj, features, labels = data.adj, data.features, data.labels

adjacency_matrix = data.adj

# Create a NetworkX graph from the adjacency matrix
graph = nx.from_scipy_sparse_array(adjacency_matrix)

# Add labels to the graph
for node_id, label in enumerate(labels):
    graph.nodes[node_id]['label'] = label
print(f"lenght of the labels : {len(labels)}") 

Loading polblogs dataset...
lenght of the labels : 1490


In [ ]:
# draw the graph  / False = without labels
nx.draw(graph, with_labels=True)
plt.show()

In [ ]:
# Draw the graph
pos = nx.spring_layout(graph)  # positions for all nodes
# nodes
nx.draw_networkx_nodes(graph, pos, node_size=10)
#nx.draw_circular(graph, with_labels=True)
# edges
nx.draw_networkx_edges(graph, pos)
# labels
labels = nx.get_node_attributes(graph, 'label')
nx.draw_networkx_labels(graph, pos, labels=labels)
plt.show()

In [99]:
#   grab graph's informations
print(graph.nodes())

#classifying nodes labels
#number of 0 and 1 labels are like : {0: 758, 1: 732}
label_counts = {}
for node_id, attributes in graph.nodes(data=True):
    label = attributes.get('label')
    if label in label_counts:
        label_counts[label] += 1
    else:
        label_counts[label] = 1
print(label_counts)


# Print the number of nodes and edges in the graph
print(f"Number of total nodes: {graph.number_of_nodes()}")
print(f"Number of total edges: {graph.number_of_edges()}")
#print(f"Number of self-loops: {graph.number_of_selfloops()}")
#print(f"Number of connected components: {nx.number_connected_components(graph)}")



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [ ]:
# Find the neighbors of a node
node_id = 1
def find_neighbors(graph, node_id):
    neighbors = list(graph.neighbors(node_id))
    print(f"number of neighbors of node {node_id}: {len(neighbors)}")
    print(f"Degree of node {node_id}: {graph.degree[node_id]}")
    print(f"Neighbors of node {node_id} and their labels:") 
    for neighbor in neighbors:
        print(f"Node ID: {neighbor}, Label: {graph.nodes[neighbor]['label']}")

find_neighbors(graph,node_id )

In [6]:
# initializations for backdoor attack
def target (graph):
    target_node = 6 
    budget = 200
    target_label = graph.nodes[target_node]['label']
    print(f"Target node is {target_node} with label {target_label} and budget {budget}")
    return target_node, target_label, budget
target(graph)


Target node is 6 with label 0 and budget 200


(6, 0, 200)

In [101]:
# initializations for backdoor attack
target_node = node_id
budget = 5
target_label = graph.nodes[target_node]['label']
print(f"Label of target node {target_node}: {target_label}")



# or define function 
# def initialization (graph):
#     target_node = 1
#     budget = 5
#     target_label = graph.nodes[target_node]['label']
#     return target_node, target_label, budget

Label of target node 1: 0


In [ ]:
#find the neighbor nodes with same label for target_node - not needed for backdoor attack
def find_same_label_neighbors(graph, target_node, target_label):
    neighbor_same = []
    for node in graph.neighbors(target_node):
        if graph.nodes[node]['label'] == target_label:
            neighbor_same.append(node)
    return neighbor_same

neighbor_same = find_same_label_neighbors(graph, target_node, target_label)
print(f"number of neighbors with same label of target {target_node}: {len(neighbor_same)}")
print(f"neighbor nodes with same label of target {target_node} : {neighbor_same}")



In [ ]:
#find the neighbor nodes with opposit label for target_node - not needed for backdoor attack
def find_opposit_label_neighbors(graph, target_node, target_label):
    neighbor_opposit = []
    for node in graph.neighbors(target_node):
        if graph.nodes[node]['label'] != target_label:
            neighbor_opposit.append(node)
    return neighbor_opposit

neighbor_opposit = find_opposit_label_neighbors(graph, target_node, target_label)
print(f"number of neighbors with opposit label of target {target_node}: {len(neighbor_opposit)}")
print(f"neighbor nodes with opposit label of target {target_node} : {neighbor_opposit}")


In [ ]:
#find the non_neighbor nodes with same label for target_node - not needed for backdoor attack
def find_non_neighbor_same_label(graph, target_node, target_label):
    non_neighbor_same = []
    for node in graph.nodes():
        if graph.nodes[node]['label'] == target_label and not graph.has_edge(target_node, node):
            non_neighbor_same.append(node)
    return non_neighbor_same

non_neighbor_same = find_non_neighbor_same_label(graph, target_node, target_label)
print(f"number of non neighbor nodes with same label of target {target_node} : {len(non_neighbor_same)}")
print(f"non neighbor nodes with same label of target {target_node} : {non_neighbor_same}") 


In [102]:
#find the non_neighbor nodes with opposit label for target_node - needed for backdoor attack
def find_non_neighbor_opposit_label(graph, target_node, target_label):
    non_neighbor_opposit = []
    for node in graph.nodes():
        if graph.nodes[node]['label'] != target_label and not graph.has_edge(target_node, node):
            non_neighbor_opposit.append(node)
    return non_neighbor_opposit

non_neighbor_opposit = find_non_neighbor_opposit_label(graph, target_node, target_label)
print(f"number of non neighbor nodes with opposit label of target {target_node} : {len(non_neighbor_opposit)}")
print(f"non neighbor nodes with opposit label of target {target_node} : {non_neighbor_opposit}") 


number of non neighbor nodes with opposit label of target 1 : 731
non neighbor nodes with opposit label of target 1 : [758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 9

In [2]:
############################## BASIC ##################################
# find the non_neighbor nodes with opposit label for target_nodes_ARRAY - needed for backdoor attack
def find_non_neighbor_opposit_label(graph, target_node):
    non_neighbor_opposit = []
    target_label = graph.nodes[target_node]['label']
    for node in graph.nodes():
        if graph.nodes[node]['label'] != target_label and not graph.has_edge(target_node, node):
            non_neighbor_opposit.append(node)
    return non_neighbor_opposit

# how to call
#non_neighbors = find_non_neighbor_opposit_label(graph, idx_test_attack, target_label)

In [103]:
#find sum of nodes neighbors 
print(f"sum of non_neighbors opposit class + non_neighbors same class: {len(non_neighbor_opposit) + len(non_neighbor_same)}")
print(f"sum of neighbors opposit class + neighbors same class): {len(neighbor_opposit) + len(neighbor_same)}")


sum of non_neighbors opposit class + non_neighbors same class: 1445
sum of neighbors opposit class + neighbors same class): 45


In [ ]:
# excel and csv writing
# Convert the outputs to dataframes
df1 = pd.DataFrame(neighbor_same, columns=["Node ID"])
df2 = pd.DataFrame(neighbor_opposit, columns=["Node ID"])
df3 = pd.DataFrame(non_neighbor_same, columns=["Node ID"])
df4 = pd.DataFrame(non_neighbor_opposit, columns=["Node ID"])
# Write the dataframes to separate sheets of an Excel file
with pd.ExcelWriter('output.xlsx') as writer:
    df1.to_excel(writer, sheet_name='neighbor_class_same', index=False)
    df2.to_excel(writer, sheet_name='neghibor_class_opposit', index=False)
    df3.to_excel(writer, sheet_name='non_neighbor_class_same', index=False)
    df4.to_excel(writer, sheet_name='non_neighbor_class_opposit', index=False)
# write to csv 
# with open('output.csv', 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(["Node ID", "Label"])  
#     for neighbor in neighbors:
#         writer.writerow([neighbor, graph.nodes[neighbor]['label']])


In [ ]:
#read from csv - excel
df = pd.read_excel('output.xlsx', sheet_name='neighbor_class_same')
print(df)

df = pd.read_csv('output.csv')
print(df)

In [105]:
############################## BASIC ##################################
#find les y most fort = plus de voisins y et moins de voisin x: at the same time
# we pass "non_neighbor_opposit" as argument of this:

def find_max_same_min_opposit_label_neighbors(graph, non_neighbor_opposit ):
  
    max_same_min_opposit_label_neighbors = []
    for node in non_neighbor_opposit:
            num_same_neighbors = sum(1 for neighbor in graph.neighbors(node) if graph.nodes[neighbor]['label'] == graph.nodes[node]['label'])
            num_opposit_neighbors = sum(1 for neighbor in graph.neighbors(node) if graph.nodes[neighbor]['label'] != graph.nodes[node]['label'])

            total_neighbors = num_opposit_neighbors + num_same_neighbors
            p = num_same_neighbors/ total_neighbors if total_neighbors != 0 else 0
            # if number of y neighbors are more than x neighbors we accept the node:
            max_same_min_opposit_label_neighbors.append((node,num_same_neighbors,p))  

            # p = num_same_neighbors/ num_opposit_neighbors if num_opposit_neighbors != 0 else 0
            # if p > 1 :
            #     max_same_min_opposit_label_neighbors.append((node,num_same_neighbors,p))  

    #max_same_min_opposit_label_neighbors.sort(key=lambda x: x[2], reverse=True)
    max_same_min_opposit_label_neighbors.sort(key=lambda x: (x[2], x[1]), reverse=True)
    return [(node, p) for node, _ ,p in max_same_min_opposit_label_neighbors]



In [106]:
max_same_min_opposit_label_neighbors= find_max_same_min_opposit_label_neighbors(graph, non_neighbor_opposit)
print("we find non neighbors nodes with opposit label of the target, then in this array")
print(f"we search for nodes with max same label and min opposit label neighbors all over the main graph: {max_same_min_opposit_label_neighbors}")


we find non neighbors nodes with opposit label of the target, then in this array
we search for nodes with max same label and min opposit label neighbors all over the main graph: [(1407, 1.0), (1149, 1.0), (951, 1.0), (816, 1.0), (1052, 1.0), (1068, 1.0), (766, 1.0), (1090, 1.0), (970, 1.0), (1454, 1.0), (853, 1.0), (1410, 1.0), (1373, 1.0), (1389, 1.0), (1194, 1.0), (1398, 1.0), (791, 1.0), (1320, 1.0), (1292, 1.0), (1191, 1.0), (1363, 1.0), (1413, 1.0), (842, 1.0), (987, 1.0), (1012, 1.0), (920, 1.0), (1333, 1.0), (1345, 1.0), (813, 1.0), (1035, 1.0), (771, 1.0), (948, 1.0), (1087, 1.0), (1039, 1.0), (1243, 1.0), (1338, 1.0), (1132, 1.0), (1401, 1.0), (1422, 1.0), (1008, 1.0), (1226, 1.0), (838, 1.0), (1217, 1.0), (1474, 1.0), (871, 1.0), (915, 1.0), (1114, 1.0), (1219, 1.0), (1286, 1.0), (1300, 1.0), (1368, 1.0), (1406, 1.0), (1437, 1.0), (929, 1.0), (1009, 1.0), (903, 1.0), (936, 1.0), (1014, 1.0), (1279, 1.0), (1430, 1.0), (1431, 1.0), (1448, 1.0), (1488, 1.0), (981, 1.0), (1023, 1

In [107]:
# Bypass CryptoGraph
# find common neighbors between 2 nodes
def find_common_neighbors(graph, node1, node2):
    neighbors1 = set(graph.neighbors(node1))
    neighbors2 = set(graph.neighbors(node2))
    common_neighbors = neighbors1.intersection(neighbors2)
    return common_neighbors


In [109]:
############################## BASIC ##################################
# Bypass CryptoGraph
# find more importent nodes + bypass cryptograph
# we pass "max_same_min_opposit_label_neighbors" as argument of this:

def nodes_for_attack(graph, target_node, max_same_min_opposit_label_neighbors, budget ):
  
    nodes_for_attack = []
    for node , p in max_same_min_opposit_label_neighbors:
            common = find_common_neighbors(graph, target_node, node)
            len_common = len(common)
            nodes_for_attack.append((node,p, len_common))
    
    nodes_for_attack.sort(key=lambda x: (x[2], x[1]), reverse=True)
    return [(node) for node in nodes_for_attack[:budget]]
    #return [(node, p) for node, p in nodes_for_attack[:budget]]

nodes_for_attack= nodes_for_attack(graph, target_node, max_same_min_opposit_label_neighbors, budget)
print(f"nodes for attack: {nodes_for_attack}")


nodes for attack: [(1438, 0.5217391304347826, 11), (1050, 0.826797385620915, 10), (797, 0.5944055944055944, 8), (1460, 0.8768115942028986, 7), (1244, 0.8834080717488789, 6)]


In [110]:
# find common neighbors between 2 nodes and test it
node1 = target_node
node2 = nodes_for_attack[0][0]
common_neighbors = find_common_neighbors(graph, target_node, node2)
print(f"Common neighbors between target node {target_node} and {node2} : {common_neighbors}")
print(f"Number of common neighbors between target node {target_node} and  {node2} : {len(common_neighbors)}")

print(f"neighbors of target node: {list(graph.neighbors(target_node))}")
print(f"neighbors of chosen node : {list(graph.neighbors(node2))}")

Common neighbors between target node 1 and 1438 : {640, 641, 98, 643, 362, 171, 686, 179, 54, 153, 154}
Number of common neighbors between target node 1 and  1438 : 11
neighbors of target node: [0, 17, 23, 28, 51, 54, 98, 143, 153, 154, 158, 171, 179, 203, 237, 247, 322, 331, 362, 388, 390, 430, 445, 476, 482, 494, 543, 546, 560, 562, 563, 565, 571, 622, 640, 641, 643, 649, 658, 663, 668, 686, 753, 755, 1130]
neighbors of chosen node : [54, 98, 114, 118, 149, 153, 154, 171, 179, 188, 346, 362, 393, 404, 433, 489, 491, 640, 641, 643, 686, 740, 781, 797, 809, 817, 855, 882, 935, 962, 1040, 1048, 1050, 1062, 1085, 1111, 1190, 1231, 1244, 1249, 1269, 1305, 1329, 1417, 1462, 1477]


In [111]:
# check if 2 nodes are neighbors before attack
node1 = target_node
node2 = nodes_for_attack[0][0]
if graph.has_edge(node1, node2):
    print(f"Before Attack: There is a link between {node1} and {node2}")
else:
    print(f"Before Attack: There is no link between {node1} and {node2}")


Before Attack: There is no link between 1 and 1438


In [112]:
############################## BASIC ##################################
#attack V_0 Mahsa
def insert_edge(graph, target_node, nodes_for_attack): 
    attacked_graph = graph.copy()
    for node, _ , _ in nodes_for_attack:
       attacked_graph.add_edge(target_node, node)
    return attacked_graph

attacked_graph = insert_edge(graph, target_node, nodes_for_attack)


In [ ]:
# check for performance of the attack if it works 
if attacked_graph.has_edge(node1, node2):
    print(f"After attack, There is a link between nodes {node1} and {node2}")
else:
    print(f"After attack, There is still no link between nodes {node1} and {node2}")

print(f"Number of total nodes in the initial graph:: {graph.number_of_nodes()}")
print(f"Number of total nodes in the initial graph:: {graph.number_of_edges()}")
print(f"Number of total nodes in the attacked graph:: {attacked_graph.number_of_nodes()}")
print(f"Number of total edges in the attacked graph:: {attacked_graph.number_of_edges()}")

if (attacked_graph.number_of_edges()- graph.number_of_edges()) == budget:
    print("Attack is successful.... here is an overview from attacked node and graph: ")

find_neighbors(attacked_graph,node_id )

In [1]:
# evaluate function for the attack
def evaluate_graph(attacked_graph, nodes_for_attack, target_node, budget):
    node1 = target_node
    insert_edge = 0
    for node2, _, _ in nodes_for_attack:
        if attacked_graph.has_edge(node1, node2):
            insert_edge += 1
    if insert_edge == budget:
        print(f"Attack is successful and {insert_edge} edges has been inserted between {target_node} and : {nodes_for_attack}")
    else:    
        print(f"Attack is not successful and {insert_edge} edges has been inserted")       

evaluate_graph(attacked_graph, nodes_for_attack, target_node, budget)
  

    # # Find the neighbors of the target node in the original graph
    # neighbors = list(graph.neighbors(target_node))
    # # Find the neighbors of the target node in the attacked graph
    # attacked_neighbors = list(attacked_graph.neighbors(target_node))
    # # Find the neighbors of the target node that are not neighbors in the original graph
    # new_neighbors = set(attacked_neighbors) - set(neighbors)
    # # Find the neighbors of the target node that are neighbors in the original graph
    # common_neighbors = set(attacked_neighbors).intersection(set(neighbors))
    # # Find the neighbors of the target node that are neighbors in the original graph
    # removed_neighbors = set(neighbors) - set(attacked_neighbors)

    

NameError: name 'attacked_graph' is not defined

In [ ]:
def convert(attacked_graph):
    # Convert the graph to a CSR matrix
    adjacency_matrix = nx.adjacency_matrix(attacked_graph)
    # Convert the adjacency matrix to CSR format
    adjacency_matrix_csr = csr_matrix(adjacency_matrix)
    return adjacency_matrix_csr

adjacency_matrix_csr = convert(attacked_graph)